<a href="https://colab.research.google.com/github/Pety99/food101/blob/main/food101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U tensorflow_datasets

Requirement already up-to-date: tensorflow_datasets in /usr/local/lib/python3.6/dist-packages (3.2.1)


In [2]:
import tensorflow as tf

In [3]:
# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

# Adathaémaz letöltése a TensorFlow Datasets-ről
A [food101](https://www.tensorflow.org/datasets/catalog/food101) adathalmazt fogjuk használni, amiben 101000 kép található 101 külöböző kategóriába tartozó ételről, kategóriánként 1000 képpel. Az adathalmazt elszeparáltuk egy train és egy validációs halmazra.


In [5]:
dataset, metadata = tfds.load('food101', as_supervised=True, with_info=True)
train_dataset, validation_dataset = dataset['train'], dataset['validation']

In [9]:
num_classes = metadata.features['label'].num_classes
num_train_examples = metadata.splits['train'].num_examples
num_val_examples = metadata.splits['validation'].num_examples


print('Total Number of Classes: {}'.format(num_classes))
print("Number of training examples: {}".format(num_train_examples))
print("Number of validation examples:     {}".format(num_val_examples))

Total Number of Classes: 101
Number of training examples: 75750
Number of validation examples:     25250


In [28]:
for i, example in enumerate(train_dataset.take(5)):
  print('Image {} shape: {} label: {}'.format(i+1, example[0].shape, example[1]))


Image 1 shape: (512, 512, 3) label: 56
Image 2 shape: (384, 512, 3) label: 76
Image 3 shape: (512, 512, 3) label: 21
Image 4 shape: (512, 512, 3) label: 64
Image 5 shape: (512, 512, 3) label: 10


In [27]:
class_names = np.array(metadata.features['label'].names)

print(class_names)

['apple_pie' 'baby_back_ribs' 'baklava' 'beef_carpaccio' 'beef_tartare'
 'beet_salad' 'beignets' 'bibimbap' 'bread_pudding' 'breakfast_burrito'
 'bruschetta' 'caesar_salad' 'cannoli' 'caprese_salad' 'carrot_cake'
 'ceviche' 'cheesecake' 'cheese_plate' 'chicken_curry'
 'chicken_quesadilla' 'chicken_wings' 'chocolate_cake' 'chocolate_mousse'
 'churros' 'clam_chowder' 'club_sandwich' 'crab_cakes' 'creme_brulee'
 'croque_madame' 'cup_cakes' 'deviled_eggs' 'donuts' 'dumplings' 'edamame'
 'eggs_benedict' 'escargots' 'falafel' 'filet_mignon' 'fish_and_chips'
 'foie_gras' 'french_fries' 'french_onion_soup' 'french_toast'
 'fried_calamari' 'fried_rice' 'frozen_yogurt' 'garlic_bread' 'gnocchi'
 'greek_salad' 'grilled_cheese_sandwich' 'grilled_salmon' 'guacamole'
 'gyoza' 'hamburger' 'hot_and_sour_soup' 'hot_dog' 'huevos_rancheros'
 'hummus' 'ice_cream' 'lasagna' 'lobster_bisque' 'lobster_roll_sandwich'
 'macaroni_and_cheese' 'macarons' 'miso_soup' 'mussels' 'nachos'
 'omelette' 'onion_rings' 'oy